## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    models=["event_emitting_transactions_list"],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-12-13 00:14:50 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=25 process=8769
2024-12-13 00:14:50 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=8769
2024-12-13 00:14:50 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76 process=8769
2024-12-13 00:14:50 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=37 process=8769
2024-12-13 00:14:50 [info     ] prepared 1 input batches.      filename=reader_bydate.py lineno=97 process=8769


## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

execute_model_in_memory(
    duckdb_client=duckdb_client,
    model="event_emitting_transactions_list",
    data_reader=batch,
    limit_input_parquet_files=1
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-12-13 00:14:50 [info     ] Executing model...             filename=testutils.py lineno=220 process=8769
2024-12-13 00:14:50 [info     ] duckdb dataset='ingestion/logs_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/logs_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=8769
2024-12-13 00:14:53 [info     ] registered view: 'ingestion_logs_v1' using 1 parquet paths filename=client.py lineno=53 process=8769
2024-12-13 00:14:53 [info     ] duckdb dataset='ingestion/transactions_v1' using 1/22 parquet paths, first path is gs://oplabs-tools-data-sink/ingestion/transactions_v1/chain=op/dt=2024-11-18/000128144000.parquet filename=reader.py lineno=68 process=8769
2024-12-13 00:14:54 [info     ] registered view: 'ingestion_transactions_v1' using 1 parquet paths filename=client.py lineno=53 process=8769
2024-12-13 00:14:54 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=8769 template=eve

┌─────────────────────────────────────┐
│                name                 │
│               varchar               │
├─────────────────────────────────────┤
│ event_emitting_transactions_list    │
│ event_emitting_transactions_list_v1 │
│ ingestion_logs_v1                   │
│ ingestion_transactions_v1           │
└─────────────────────────────────────┘

## Verify model results

In [3]:
duckdb_client.sql("SELECT * FROM event_emitting_transactions_list_v1 LIMIT 10")

┌────────────┬─────────────────────┬─────────────────┬─────────┬─────────┬──────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────┬────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┐
│     dt     │     block_hour      │ block_timestamp │ network │  chain  │ chain_id │ block_number │                          transaction_hash                          │ gas_price  │ count_total_events │ count_approval_events │ count_wrapping_events │ count_transfer_events │
│    date    │      timestamp      │     uint32      │ varchar │ varchar │  int32   │    int64     │                              varchar                               │   int64    │       int64        │     decimal(38,0)     │     decimal(38,0)     │     decimal(38,0)     │
├────────────┼─────────────────────┼─────────────────┼─────────┼─────────┼──────────┼──────────────┼────────────────────────────────────────────────────────────────────┼───

### Check the data output size

In [4]:
duckdb_client.sql("SELECT COUNT(*) FROM event_emitting_transactions_list_v1")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        16679 │
└──────────────┘

In [6]:
duckdb_client.sql("SELECT input FROM event_emitting_transactions_list_v1 LIMIT 10")


BinderException: Binder Error: Referenced column "input" not found in FROM clause!
Candidate bindings: "event_emitting_transactions_list_v1.network", "event_emitting_transactions_list_v1.chain_id", "event_emitting_transactions_list_v1.block_number"

### You can also convert the results to dataframes to inspect them in more familiar ways

In [ ]:
duckdb_client.sql("SELECT * FROM refined_transactions_fees_v1 LIMIT 10").pl().head()

### Get table schema

In [ ]:
duckdb_client.sql("DESCRIBE event_emitting_transactions_v1")
